# This notebook helps us visualize the model's performance
I have gathered the following data from the model during training:
- learning rates
- embeddings
- probabilities
- train and validation losses
- train and validation perplexities



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
folder_path = '/content/drive/MyDrive/Colab Notebooks/MIDI-gen-notebooks/sample_rock_set'
proj_folder_path = '/content/drive/MyDrive/Colab Notebooks/MIDI-gen-notebooks/'
FULL_MIDI_LANG_DATA_path = '/content/drive/MyDrive/Colab Notebooks/MIDI-gen-notebooks/Large_MIDI_Language_Base.parquet'
learning_rates_parq = "/content/drive/MyDrive/Colab Notebooks/MIDI-gen-notebooks/training_data/learning_rates_df.parquet"
train_losses_parq = "/content/drive/MyDrive/Colab Notebooks/MIDI-gen-notebooks/training_data/train_losses_df.parquet"
train_ppl_parq = "/content/drive/MyDrive/Colab Notebooks/MIDI-gen-notebooks/training_data/train_ppl_df.parquet"
val_losses_parq = "/content/drive/MyDrive/Colab Notebooks/MIDI-gen-notebooks/training_data/val_losses_df.parquet"
val_ppl_parq = "/content/drive/MyDrive/Colab Notebooks/MIDI-gen-notebooks/training_data/val_ppl_df.parquet"
attention_weights_parq="/content/drive/MyDrive/Colab Notebooks/MIDI-gen-notebooks/training_data/attention_weights_df.parquet"
latest_model = "/content/drive/MyDrive/Colab Notebooks/MIDI-gen-notebooks/07-12-23_23:06_model_weights.pth"

In [5]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from music21 import chord, note, stream, clef, meter,converter
import os
from tqdm import tqdm
import datetime
from IPython.display import clear_output, Image, display
import PIL
import pandas as pd
import seaborn as sns
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:100'

In [4]:
FULL_MIDI_LANG_DATA_list = pd.read_parquet(FULL_MIDI_LANG_DATA_path)["full_MIDI_lang_base"].values.tolist()
learning_rates = pd.read_parquet(learning_rates_parq)["learning_rates"].values.tolist()
train_losses = pd.read_parquet(train_losses_parq)["train_losses"].values.tolist()
val_losses = pd.read_parquet(val_losses_parq)["val_losses"].values.tolist()
train_ppl = pd.read_parquet(train_ppl_parq)["train_ppl"].values.tolist()
val_ppl = pd.read_parquet(val_ppl_parq)["val_ppl"].values.tolist()
attention_weights = pd.read_parquet(attention_weights_parq)["attention_weights"].values.tolist()

# Visualize learning rates

In [10]:
import plotly.graph_objects as go

def plot_learning_rate(learning_rates):
    # if len(learning_rates) > 200:
    #     fig = go.Figure(data=go.Scatter(x=list(range(len(learning_rates[-200:]))), y=learning_rates[-200:], mode='lines'))
    # else:
    #     fig = go.Figure(data=go.Scatter(x=list(range(len(learning_rates))), y=learning_rates, mode='lines'))
    fig = go.Figure(data=go.Scatter(x=list(range(len(learning_rates))), y=learning_rates, mode='lines'))
    fig.update_layout(
        title='Learning Rate over Time',
        xaxis_title='Step',
        yaxis_title='Learning Rate',
        hovermode='x unified',  # Enables a vertical line at the point of hover and displays the value.
        hoverlabel=dict(  # Overrides default formatting for hover label
            namelength=-1,  # To display full variable name
            bgcolor="white",  # Background color
            font_size=15,  # Font size
            font_family="Rockwell"  # Font
        )
    )
    fig.update_yaxes(hoverformat=".13f")  # Updates the y-axis hover format to three decimal places.

    fig.show()

# learning_rates = [0.01, 0.02, 0.03, 0.04]  # Use your actual data
plot_learning_rate(learning_rates)



In [12]:
def plot_losses(train_losses, val_losses):
    fig = go.Figure()

    # if len(train_losses) > 200 and len(val_losses) > 200:
    #     fig.add_trace(go.Scatter(x=list(range(len(train_losses[-200:]))), y=train_losses[-200:], mode='lines', name='Train Loss', line=dict(color='blue')))
    #     fig.add_trace(go.Scatter(x=list(range(len(val_losses[-200:]))), y=val_losses[-200:], mode='lines', name='Validation Loss', line=dict(color='orange')))
    # else:
    #     fig.add_trace(go.Scatter(x=list(range(len(train_losses))), y=train_losses, mode='lines', name='Train Loss', line=dict(color='blue')))
    #     fig.add_trace(go.Scatter(x=list(range(len(val_losses))), y=val_losses, mode='lines', name='Validation Loss', line=dict(color='orange')))
    fig.add_trace(go.Scatter(x=list(range(len(train_losses))), y=train_losses, mode='lines', name='Train Loss', line=dict(color='blue')))
    fig.add_trace(go.Scatter(x=list(range(len(val_losses))), y=val_losses, mode='lines', name='Validation Loss', line=dict(color='orange')))
    fig.update_layout(
        title='Train and Validation Loss over Time',
        xaxis_title='Step',
        yaxis_title='Loss',
        hovermode='x unified',
        hoverlabel=dict(
            namelength=-1,
            bgcolor="white",
            font_size=15,
            font_family="Rockwell"
        )
    )
    fig.update_yaxes(hoverformat=".4f")
    fig.show()


plot_losses(train_losses, val_losses)
